## To Calculate and Save Spectrograms

    Authors: Fernanda Ferreira and Victor Paganotto
    Professor: Tiago F. Tavares
    
    date: Sept/2019

#### Import Libraries

In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import glob
import librosa
import librosa.display as ld
# Printa todos os valores do array
#np.set_printoptions(threshold=np.inf)
np.set_printoptions(threshold=10)
import pandas as pd
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import keras
import os
import re
from sklearn.externals import joblib

#### To load Functions

In [6]:
def read_audios(data, n): #1, 0,2 0,5-0,75
    len_audios = []
    audios = []

    for file in data:
        #input()
        #audio, sr = librosa.core.load(file, sr=22050)
        audio, sr = librosa.load(file, sr=22050)
        audio = (audio - np.mean(audio))/np.std(audio)
        #print (len(audio))
        maxN = int(n * sr)
        x = audio[0:maxN]
        x = (x - np.mean(x)) / np.std(x)
        w = np.hanning(len(x))
        xw = x * w
        audios.append(xw)
        len_audios.append(len(xw))
        
    max_audios = max(len_audios)
        
    return audios, max_audios

#To convert the hop length and frame size to units of seconds:
#print ("hop length[s]:", float(hop_length)/sr) # units of seconds
#print ("frame size[s]:",float(n_fft)/sr) # units of seconds

In [7]:
def my_specs(audios, max_audios):
    specs = []
    
    for i in range(len(audios)):
        i_audio = np.array(audios[i])
        i_audio.resize(max_audios)
        x = librosa.feature.melspectrogram(y=i_audio, sr=22050, S=None, n_fft=2048, hop_length=1024)
        X = np.abs(x)
        specs.append(X)
        
    return specs


In [8]:
def count_elements(seq) -> dict:
    """Tally elements from `seq`."""
    hist = {}
    for i in seq:
        hist[i] = hist.get(i, 0) + 1
    return hist

In [36]:
def read_labels(data):
    l = []
    for file in data:
        (dirname, sname) = os.path.split(file)
        (name, ext) = os.path.splitext(sname)
        (label, num) = os.path.splitext(name)
        n = re.split('-', label, flags=re.IGNORECASE)
        n2 = re.split('[0-9]+', n[0], flags=re.IGNORECASE)
        if len(n2)<2:
            l.append(n2[0])
            #print(n2[0])
        else:
            l.append(n2[1])
            #print(n2[1])
    return l

In [6]:
# convert class vectors to binary class matrices
# def string_to_binary(lista):
    
#     for i in range(len(lista)):
#         if lista[i] == 'P':
#             lista[i] = 0
#         elif lista[i] == 'NP':
#             lista[i] = 1
#     return lista

In [25]:
def main(data_train, data_test, n):
    audios_train, max_audios_train = read_audios(data_train, n)
    audios_test, max_audios_test = read_audios(data_test, n)
    
    if max_audios_train > max_audios_test:
        specs_train = my_specs(audios_train, max_audios_train)
    else:
        specs_train = my_specs(audios_train, max_audios_test)
        
    if max_audios_train > max_audios_test:
        x_test = my_specs(audios_test, max_audios_train)
    else:
        x_test = my_specs(audios_test, max_audios_test)
               
    l_train = read_labels(data_train)
    y_test = read_labels(data_test)
    
    l_count_train = count_elements(l_train)
    l_count_test = count_elements(y_test)
    print('Train:', l_count_train, 'Test:', l_count_test)
    
    #stratify: certifica a mesma quantidade de audios de cada classe.
    x_train, x_val, y_train, y_val = train_test_split(specs_train, l_train, test_size=0.2, stratify=l_train)
    
    #y_train_bi = string_to_binary(y_train)
    #print (y_train_bi)

    #y_test_bi = string_to_binary(y_test)
    #print (y_test_bi)

    #y_val_bi = string_to_binary(y_val)
    #print (y_val_bi)
    
    x_train_np = np.array(x_train)
    x_test_np = np.array(x_test)
    x_val_np = np.array(x_val)

    y_train_np = np.array(y_train)
    y_test_np = np.array(y_test)
    y_val_np = np.array(y_val)

    print (x_train_np.shape, y_train_np.shape)
    print (x_test_np.shape, y_test_np.shape)
    print(x_val_np.shape, y_val_np.shape)
    
    return x_train_np, x_test_np, x_val_np, y_train_np, y_test_np, y_val_np


In [15]:
# ------------------------- Completa -----------------------------------
# data_train = glob.glob('')
# print (len(data_train))
# data_test = glob.glob('Novos_testes/Completa/Test/*.mp3')
# print (len(data_test))

# ------------------------- Trans_trans -----------------------------------
# data_train = glob.glob('Novos_testes/T_T/Train/*.mp3')
# print (len(data_train))
# data_test = glob.glob('Novos_testes/T_T/Test/*.mp3')
# print (len(data_test))

#-------------------------- NoTrans_trans --------------------------------
#data_train = glob.glob('Novos_testes/NT_T/Train/*.mp3')
#print (len(data_train))
#data_test = glob.glob('Novos_testes/NT_T/Test/*.mp3')
#print (len(data_test))

#---------------------------- Trans_NoTrans -------------------------------
#data_train = glob.glob('Novos_testes/T_NT/Train/*.mp3')
#print (len(data_train))
#data_test = glob.glob('Novos_testes/T_NT/Test/*.mp3')
#print (len(data_test))

data_train = glob.glob('Generos/Data_driven/Train/*')
print (len(data_train))
data_test = glob.glob('Generos/Data_driven/Test/*')
print (len(data_test))

550
548


In [37]:
# 1 -> 5seg | 0.2 -> 1seg | 0.75 -> 0.75seg | 0.5 -> 0.5seg
N = [10, 5, 2, 1]

for n in N:
    x_train_np, x_test_np, x_val_np, y_train_np, y_test_np, y_val_np = main(data_train, data_test, n)
    
    joblib.dump(x_train_np, "Generos/Data_driven/features/x_train"+str(n)+".pkl")
    joblib.dump(x_test_np, "Generos/Data_driven/features/x_test"+str(n)+".pkl")
    joblib.dump(x_val_np, "Generos/Data_driven/features/x_val"+str(n)+".pkl")
    joblib.dump(y_train_np, "Generos/Data_driven/features/y_train"+str(n)+".pkl")
    joblib.dump(y_test_np, "Generos/Data_driven/features/y_test"+str(n)+".pkl")
    joblib.dump(y_val_np, "Generos/Data_driven/features/y_val"+str(n)+".pkl")


Train: {'disco': 50, 'pop': 50, 'metal': 50, 'jazz': 50, 'P': 50, 'classical': 50, 'rock': 50, 'hiphop': 50, 'country': 50, 'blues': 50, 'reggae': 50} Test: {'pop': 50, 'disco': 50, 'blues': 50, 'metal': 50, 'reggae': 50, 'jazz': 50, 'rock': 50, 'P': 50, 'country': 50, 'hiphop': 48, 'classical': 50}
(440, 128, 216) (440,)
(548, 128, 216) (548,)
(110, 128, 216) (110,)
Train: {'disco': 50, 'pop': 50, 'metal': 50, 'jazz': 50, 'P': 50, 'classical': 50, 'rock': 50, 'hiphop': 50, 'country': 50, 'blues': 50, 'reggae': 50} Test: {'pop': 50, 'disco': 50, 'blues': 50, 'metal': 50, 'reggae': 50, 'jazz': 50, 'rock': 50, 'P': 50, 'country': 50, 'hiphop': 48, 'classical': 50}
(440, 128, 108) (440,)
(548, 128, 108) (548,)
(110, 128, 108) (110,)
Train: {'disco': 50, 'pop': 50, 'metal': 50, 'jazz': 50, 'P': 50, 'classical': 50, 'rock': 50, 'hiphop': 50, 'country': 50, 'blues': 50, 'reggae': 50} Test: {'pop': 50, 'disco': 50, 'blues': 50, 'metal': 50, 'reggae': 50, 'jazz': 50, 'rock': 50, 'P': 50, 'coun

#### Save features

In [16]:
# --------------------------- Completa -------------------------------------------------------------
# joblib.dump(x_train_np, "Escala_Mel/Data_driven/Completa/05seg/x_train.pkl")
# joblib.dump(x_test_np, "Escala_Mel/Data_driven/Completa/05seg/x_test.pkl")
# joblib.dump(x_val_np, "Escala_Mel/Data_driven/Completa/05seg/x_val.pkl")
# joblib.dump(y_train_np, "Escala_Mel/Data_driven/Completa/05seg/y_train.pkl")
# joblib.dump(y_test_np, "Escala_Mel/Data_driven/Completa/05seg/y_test.pkl")
# joblib.dump(y_val_np, "Escala_Mel/Data_driven/Completa/05seg/y_val.pkl")

#-------------------- Trans_Trans-------------------------------------------------------------------
#joblib.dump(x_train_np, "Escala_Mel/Data_driven/Trans_Trans/05seg/x_train.pkl")
#joblib.dump(x_test_np, "Escala_Mel/Data_driven/Trans_Trans/05seg/x_test.pkl")
#joblib.dump(x_val_np, "Escala_Mel/Data_driven/Trans_Trans/05seg/x_val.pkl")
#joblib.dump(y_train_np, "Escala_Mel/Data_driven/Trans_Trans/05seg/y_train.pkl")
#joblib.dump(y_test_np, "Escala_Mel/Data_driven/Trans_Trans/05seg/y_test.pkl")
#joblib.dump(y_val_np, "Escala_Mel/Data_driven/Trans_Trans/05seg/y_val.pkl")

#-------------------------- NoTrans_trans -------------------------------------------------------------
#joblib.dump(x_train_np, "Escala_Mel/Data_driven/NoTrans_Trans/05seg/x_train.pkl")
#joblib.dump(x_test_np, "Escala_Mel/Data_driven/NoTrans_Trans/05seg/x_test.pkl")
#joblib.dump(x_val_np, "Escala_Mel/Data_driven/NoTrans_Trans/05seg/x_val.pkl")
#joblib.dump(y_train_np, "Escala_Mel/Data_driven/NoTrans_Trans/05seg/y_train.pkl")
#joblib.dump(y_test_np, "Escala_Mel/Data_driven/NoTrans_Trans/05seg/y_test.pkl")
#joblib.dump(y_val_np, "Escala_Mel/Data_driven/NoTrans_Trans/05seg/y_val.pkl")

# ---------------------------- Trans_NoTrans --------------------------------------------------------------
#joblib.dump(x_train_np, "Escala_Mel/Data_driven/Trans_NoTrans/05seg/x_train.pkl")
#joblib.dump(x_test_np, "Escala_Mel/Data_driven/Trans_NoTrans/05seg/x_test.pkl")
#joblib.dump(x_val_np, "Escala_Mel/Data_driven/Trans_NoTrans/05seg/x_val.pkl")
#joblib.dump(y_train_np, "Escala_Mel/Data_driven/Trans_NoTrans/05seg/y_train.pkl")
#joblib.dump(y_test_np, "Escala_Mel/Data_driven/Trans_NoTrans/05seg/y_test.pkl")
#joblib.dump(y_val_np, "Escala_Mel/Data_driven/Trans_NoTrans/05seg/y_val.pkl")

['Escala_Mel/Data_driven/Completa/05seg/y_val.pkl']

### Geração da matriz Random

In [13]:
print (x_train_np.shape[1])

128


In [16]:
Rn = np.random.randn(x_train_np.shape[1], x_train_np.shape[1])
#print(Rn)
#print(Rn.shape)

#joblib.dump(Rn, "Escala_Mel/Random/Mx_random.pkl")

Rn = joblib.load("Escala_Mel/Random/Mx_random.pkl")
print (Rn.shape)

# x_train_Rn = np.zeros((x_train_np.shape[0], x_train_np.shape[1], x_train_np.shape[2]))

# for i in range(len(x_train_np)):
#     x_train_Rn[i][:][:] = np.dot(Rn, x_train_np[i])
#     #x_train_mx[i][:] = x_train_np[i] * mx
    
# print (x_train_Rn.shape)

(128, 128)
